# Лабораторная работа 1. Морфологический анализ

### Задание 1.
1. Изучить демонстрационный ноутбук `"Сегментация и токенизация"`.
1. Выберите наилучшую, на ваш взгляд, библиотеку для сегментации и токенизации.

### Задание 2.
1. Изучите документацию и лицензию морфологического парсера [pymorphy2](https://pymorphy2.readthedocs.io).
1. Установите `pymorphy2`.

### Задание 3.
Напишите функцию `parse_text()`, на вход которых поступает текст (в виде строки), а на выходе формируется список, содержащий для каждого слова входного текста следующую информацию:
- исходную словоформу (`wordform`);
- нормальную форму слова (лемму) (`norm` или `lemma`);
- часть речи (part of speech, `pos`);
- другую грамматическую информацию, выдаваемую `pymorphy2`;
- признак, присутствует ли слово в словаре `pymorphy2`.

Функция должна выбирать наиболее вероятный вариант морфологического разбора слова.


In [118]:
text0 = 'Напишите функцию parse_text(), на вход которых поступает текст (в виде строки), а на выходе формируется список, содержащий для каждого слова входного текста следующую информацию:'

In [119]:
text1 = 'Крепостного права я не знал и не видел, но помню, у тетки Анны Герасимовны чувствовал его. Въедешь во двор и сразу ощущаешь, что тут оно еще вполне живо. Усадьба – небольшая, но вся старая, прочная, окруженная столетними березами и лозинами. Надворных построек – невысоких, но домовитых – множество, и все они точно слиты из темных дубовых бревен под соломенными крышами. Выделяется величиной или, лучше сказать, длиной только почерневшая людская, из которой выглядывают последние могикане дворянского сословия – какие-то ветхие старики и старухи, дряхлый повар в отставке, похожий на Дон-Кихота. Все они, когда въезжаешь во двор, подтягиваются и низко-низко кланяются. Седой кучер, направляющийся от каретного сарая взять лошадь, еще у сарая снимает шапку и по всему двору идет с обнаженной головой. Он у тетки ездил форейтором, а теперь возит ее к обедне, - зимой в возке, а летом в крепкой, окованной железом тележке, вроде тех, на которых ездят попы. Сад у тетки славился своей запущенностью, соловьями, горлинками и яблоками, а дом – крышей. Стоял он во главе двора, у самого сада, - ветки лип обнимали его, - был невелик и приземист, но казалось, что ему и веку не будет, - так основательно глядел он из-под своей необыкновенно высокой и толстой соломенной крыши, почерневшей и затвердевшей от времени. Мне его передний фасад представлялся всегда живым: точно старое лицо глядит из-под огромной шапки впадинами глаз, - окнами с перламутровыми от дождя и солнца стеклами.'

In [186]:
text2 = 'дождище душещипательская истоия дождь кстати еще дождик машина машинка машинное душещипательская земля земельный идет дождь школа и сервис отца с дождём и огнём воды сразу с обрыва в облака облачные шторм шёл назад и вперёд'

In [121]:
import stanza
import pymorphy2
import logging
import re
import csv
from pprint import pprint
import copy
import json

In [122]:
stanza.download("ru")

2021-10-13 22:02:04 INFO: Downloading default packages for language: ru (Russian)...
INFO:stanza:Downloading default packages for language: ru (Russian)...
2021-10-13 22:02:05 INFO: File exists: C:\Users\artem\stanza_resources\ru\default.zip.
INFO:stanza:File exists: C:\Users\artem\stanza_resources\ru\default.zip.
2021-10-13 22:02:08 INFO: Finished downloading models and saved to C:\Users\artem\stanza_resources.
INFO:stanza:Finished downloading models and saved to C:\Users\artem\stanza_resources.


In [187]:
text_example = copy.copy(text2)

In [124]:
gr = {
    "NOUN":"имя существительное",
    "ADJF": "имя прилагательное (полное)",
    "NOUN":"имя существительное",
    "ADJF":"имя прилагательное (полное)",
    "ADJS":"имя прилагательное (краткое)",
    "COMP":"компаратив",
    "VERB":"глагол (личная форма)",
    "INFN":"глагол (инфинитив)",
    "PRTF":"причастие (полное)",
    "PRTS":"причастие (краткое)",
    "GRND":"деепричастие",
    "NUMR":"числительное",
    "ADVB":"наречие",
    "NPRO":"местоимение-существительное",
    "PRED":"предикатив",
    "PREP":"предлог",
    "CONJ":"союз",
    "PRCL":"частица",
    "INTJ":"междометие",
    "nomn":"именительный",
    "gent":"родительный",
    "datv":"дательный",
    "accs":"винительный",
    "ablt":"творительный",
    "loct":"предложный",
    "voct":"звательный",
    "gen2":"второй родительный (частичный)",
    "acc2":"второй винительный",
    "loc2":"второй предложный (местный)",
    "masc":"мужской род",
    "femn":"женский род",
    "neut":"средний род",
    "LATN":"Токен состоит из латинских букв (например, “foo-bar” или “Maßstab”)",
    "PNCT":"Пунктуация (например, , или !? или …)",
    "NUMB":"Число (например, “204” или “3.14”)",
    "intg":"целое число (например, “204”)",
    "real":"вещественное число (например, “3.14”)",
    "ROMN":"Римское число (например, XI)",
    "UNKN":"Токен не удалось разобрать",
}

In [209]:
ORIGIN_WORD = 'OriginForm'
NORMAL_FORM = 'NormalForm'
POS = 'POS'
CASE = 'Case'
GENDER = 'Gender'
DICT_EXIST = 'FromDict'
WORDS = "Words"
FREQ = "Freq"
UNIQ = "uniquies"
COUNT = "count"

In [154]:
def print_dict(d):
    print(json.dumps(d, indent=4, ensure_ascii=False))

In [127]:
morph = pymorphy2.MorphAnalyzer()
nlp = stanza.Pipeline(lang='ru', processors='tokenize')

2021-10-13 22:02:08 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |

INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |

2021-10-13 22:02:08 INFO: Use device: gpu
INFO:stanza:Use device: gpu
2021-10-13 22:02:08 INFO: Loading: tokenize
INFO:stanza:Loading: tokenize
2021-10-13 22:02:09 INFO: Done loading processors!
INFO:stanza:Done loading processors!


In [128]:
def get_data(t):
    return 'None' if t is None else t

def get_is_known(t):
    return True if t else False

def parse_text(text):
    ret = []
    t = nlp(text)
    tokens = []
    for sentence in t.sentences:
        tokens += [token.text for token in sentence.tokens]
    for word in tokens:
        if re.match(r'\w+', word):
            parsed = morph.parse(word)[0]
            data = {
                ORIGIN_WORD : word,
                NORMAL_FORM : parsed.normal_form,
                POS         : get_data(gr.get(parsed.tag.POS)),
                CASE        : get_data(gr.get(parsed.tag.case)) ,
                GENDER      : get_data(gr.get(parsed.tag.gender)),
                DICT_EXIST  : get_is_known(morph.word_is_known(word)),
            }
            ret.append({word : data})              
    return ret

In [129]:
parse_text(text_example)

[{'дождище': {'OriginForm': 'дождище', 'NormalForm': 'дождище', 'POS': 'имя существительное', 'Case': 'именительный', 'Gender': 'мужской род', 'FromDict': True}}, {'душещипательская': {'OriginForm': 'душещипательская', 'NormalForm': 'душещипательский', 'POS': 'имя прилагательное (полное)', 'Case': 'именительный', 'Gender': 'женский род', 'FromDict': False}}, {'истоия': {'OriginForm': 'истоия', 'NormalForm': 'истоия', 'POS': 'имя существительное', 'Case': 'именительный', 'Gender': 'женский род', 'FromDict': False}}, {'дождь': {'OriginForm': 'дождь', 'NormalForm': 'дождь', 'POS': 'имя существительное', 'Case': 'именительный', 'Gender': 'мужской род', 'FromDict': True}}, {'кстати': {'OriginForm': 'кстати', 'NormalForm': 'кстати', 'POS': 'союз', 'Case': 'None', 'Gender': 'None', 'FromDict': True}}, {'еще': {'OriginForm': 'еще', 'NormalForm': 'ещё', 'POS': 'наречие', 'Case': 'None', 'Gender': 'None', 'FromDict': True}}, {'дождик': {'OriginForm': 'дождик', 'NormalForm': 'дождик', 'POS': 'имя

### Задание 4.
Напишите функцию `save_morph_results()`, сохраняющую структуру данных, получаемую функцией `parse_text()`, в текстовый файл формата JSON.

In [130]:
def save_morph_results(d):
    with open("morph_results.json", "w+", encoding='utf-8') as f:
        tojs = json.dumps(d, indent=4, ensure_ascii=False)
        f.write(tojs)

In [188]:
save_morph_results(parse_text(text_example))

### Задание 5.
Напишите функцию `get_dictionary()`, на вход которой поступает структура данных, получаемая функцией `parse_text()`, а на выходе формируется словарь, ключами которого являются все нормальные формы слов текста, а в качестве значений хранится следующая информация:
- часть речи слова;
- частота слова в тексте;
- все варианты словоформ в тексте с данной нормальной формой.

In [149]:
# получаем все слова с такой нормальной формой
def get_words(data, normal_word):
    wrds = []
    for d in data:
        if list(d.values())[0][NORMAL_FORM]  == normal_word:
            wrds.append(list(d)[0])
    return wrds

In [201]:
def get_dictionary(parsed):
    tmp = {}
    for val in parsed:
        value = list(val.values())[0]
        wrds_list = get_words(list(parsed), value[NORMAL_FORM])
        tmp.update({value[NORMAL_FORM] : { 
            POS   : value[POS], 
            WORDS : wrds_list,
        }})
    for k, v in tmp.items():
        tmp[k].update({FREQ : round(len(v[WORDS]) / len(parsed), 4)}) 
    return tmp

In [202]:
get_dictionary(parse_text(text_example))

{'дождище': {'POS': 'имя существительное', 'Words': ['дождище'], 'Freq': 0.0286}, 'душещипательский': {'POS': 'имя прилагательное (полное)', 'Words': ['душещипательская', 'душещипательская'], 'Freq': 0.0571}, 'истоия': {'POS': 'имя существительное', 'Words': ['истоия'], 'Freq': 0.0286}, 'дождь': {'POS': 'имя существительное', 'Words': ['дождь', 'дождь', 'дождём'], 'Freq': 0.0857}, 'кстати': {'POS': 'союз', 'Words': ['кстати'], 'Freq': 0.0286}, 'ещё': {'POS': 'наречие', 'Words': ['еще'], 'Freq': 0.0286}, 'дождик': {'POS': 'имя существительное', 'Words': ['дождик'], 'Freq': 0.0286}, 'машина': {'POS': 'имя существительное', 'Words': ['машина'], 'Freq': 0.0286}, 'машинка': {'POS': 'имя существительное', 'Words': ['машинка'], 'Freq': 0.0286}, 'машинный': {'POS': 'имя прилагательное (полное)', 'Words': ['машинное'], 'Freq': 0.0286}, 'земля': {'POS': 'имя существительное', 'Words': ['земля'], 'Freq': 0.0286}, 'земельный': {'POS': 'имя прилагательное (полное)', 'Words': ['земельный'], 'Freq': 

### Задание 6.
Напишите функцию `save_dictionary()`, сохраняющую предыдущую структуру данных в текстовый файл формата JSON. Слова в файле должны быть упорядочены по убыванию частоты.

In [180]:
def save_dictionary(d):
    with open("get_dictionary_results.json", "w+", encoding='utf-8') as f:
        tojs = json.dumps(d, indent=4, ensure_ascii=False)
        f.write(tojs)

In [189]:
save_dictionary(get_dictionary(parse_text(text_example)))

### Задание 7.
Напишите функцию `get_non_dict()`, на вход которой поступает структура данных, получаемая функцией `parse_text()`, а на выходе формируется словарь, содержащий слова текста, отсутствующие в словаре `pymorphy2`, вместе с частотой слова в тексте.

In [203]:
def get_non_dict(parsed):
    d = {}
    for v in parsed:
        key = list(v)[0]
        if not v[key][DICT_EXIST]:
            count = d.get(v[key][ORIGIN_WORD], 0) + 1
            d.update({ v[key][ORIGIN_WORD] : count})
    for k, v in d.items():
        d.update({k : round(v / len(parsed), 4)}) 
    return d

In [204]:
get_non_dict(parse_text(text_example))

{'душещипательская': 0.0571, 'истоия': 0.0286}

### Задание 8.
Напишите функцию `save_non_dict()`, сохраняющую структуру данных, получаемую функцией `get_non_dict()`, в текстовый файл формата TSV (tab-separated values). Слова в файле должны быть упорядочены по убыванию частоты.

In [207]:
def save_non_dict(data):
    with open('non_dict_results.tsv', 'w', encoding='utf-8') as tsvfile:
        for k, v in data.items():
            tsvfile.write(f"{k}\t{v}\n")

In [208]:
save_non_dict(get_non_dict(parse_text(text_example)))

### Задание 9.
Напишите функцию `get_pos_distribution()`, на вход которой поступает словарь, формируемый функцией `get_dictionary()`, а на выходе выдается структура данных, содержащая частотное распределение частей речи в словаре:
- часть речи – количество уникальных слов – общее количество слов.

In [92]:
def get_pos_distribution(data):
    d = {}
    for k, v in data.items():
        if d.get(v[POS]) is None:
            d.update({v[POS] : { UNIQ : 1, COUNT : v[FREQ]}})
        else:
            tmp_u = d[v[POS]].get(UNIQ) + 1
            tmp_c = d[v[POS]].get(COUNT) + v[FREQ]
            d.update({v[POS] : { UNIQ : tmp_u, COUNT : tmp_c}})
    return d

In [93]:
get_pos_distribution(get_dictionary(parse_text(text_example)))

{'имя существительное': {'uniquies': 13, 'count': 13},
 'союз': {'uniquies': 2, 'count': 2},
 'наречие': {'uniquies': 4, 'count': 4},
 'имя прилагательное (полное)': {'uniquies': 3, 'count': 3},
 'глагол (личная форма)': {'uniquies': 1, 'count': 1},
 'предлог': {'uniquies': 2, 'count': 2}}

### Задание 10.
Создайте текстовую коллекцию - скачайте не менее **10 текстов** разных жанров и разного размера (например, произведения классиков, современных писателей, новостные статьи, научные статьи и т.п.).

Учитывайте кодировку – все файлы должны быть в UTF-8.

In [ ]:
# Ваш код для открытия файлов.

### Задание 11.
Обработайте текстовую коллекцию при помощи функций `parse_text()`, `get_dictionary()` и `get_non_dict()`, и сохраните результаты в текстовых файлах при помощи функций `save_morph_results()`, `save_dictionary()` и `save_non_dict()`.

Для каждого входного текстового файла выведите на экран следующую информацию:
- имя файла;
- размер файла в байтах;
- размер текста в словах (общее количество слов в тексте);
- размер словаря (количество уникальных слов в тексте);
- время обработки файла (в секундах).

In [ ]:
# Ваш код для обработки коллекции и вывода результатов

### Задание 12.
Для самого большого словаря выполните следующее:
- постройте частотное распределение слов: по оси ординат – частота, по оси абсцисс – слова, упорядоченные по убыванию частоты (по-другому, *ранги* слов). В качестве значений на оси абсцисс можно выводить ранги слов, а не сами слова;
- постройте в виде диаграммы распределение частей речи, полученное функцией `get_pos_distribution()`.

In [ ]:
# Ваш код для вывода графика и диаграммы

### Задание 13.
Постройте график зависимость времени морфологического анализа от размера текстового файла.

In [ ]:
# Ваш код для вывода графика